In [1]:
# Useful starting lines
%matplotlib inline

import random
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import time


%load_ext autoreload
%autoreload 2

In [2]:
from helpers_own import *

Import data -> use of the imported function made by the ML team (takes a long time)

In [4]:
from helpers import load_csv_data

x_train, x_test, y_train, train_ids, test_ids = load_csv_data(".", sub_sample=False)

This method allows us to take a specific feature out of X. My idea is to take a few of the interesting features out and concatenate them together.

In [5]:
def read_first_line(filename):
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        first_line = next(reader)
        return first_line
        
def extract_feature(name):
    filename = 'x_train.csv'
    first_line = np.array(read_first_line(filename))
    index = np.where(first_line == name)
    ind = index[0].item()
    return x_train[:, ind-1]

Here is a quick method to see that our data is not "clean". There are a lot of nan.

In [25]:
def caracteristics(x, y):
    count = np.sum(~np.isnan(x))
    print("Number of x features beeing not nan:", count)
    nan_count = np.sum(np.isnan(x))
    print("Number of x features beeing nan:", nan_count)
    num_negatives = np.sum(y == -1)
    print("Number of y -1s:", num_negatives)
    num_positives = np.sum(y == 1)
    print("Number of y 1s:", num_positives)
    num_null = np.sum(y == 0)
    print("Number of y 0s:", num_null)
    nan_count_y = np.sum(np.isnan(y))
    print("Number of y nan:", nan_count_y)

#Here is for the specific feature "_BMI5"
caracteristics(x_train,y_train)

Number of x features beeing not nan: 58177081
Number of x features beeing nan: 47154254
Number of y -1s: 299160
Number of y 1s: 28975
Number of y 0s: 0
Number of y nan: 0


Now we are ready to extract a few features and replace the lack of data with the mean (those features are taken from the website: https://medium.com/@alexteboul17/building-predictive-models-for-heart-disease-using-the-2015-behavioral-risk-factor-surveillance-b786368021ab

In [7]:
def replace_mean(x):
    mean_value = np.nanmean(x)
    x_new = x.copy()
    x_new[np.isnan(x_new)] = mean_value
    return x_new

def replace_mode(x):
    unique, counts = np.unique(x[~np.isnan(x)], return_counts=True)
    mode_value = unique[np.argmax(counts)]
    # Step 2: Replace NaN values with the mode
    x[np.isnan(x)] = mode_value
    return x


################Body mass idex - continuous feature
_BMI5 = extract_feature('_BMI5')
_BMI5 = replace_mean(_BMI5)
_BMI5, _BMI5_mean, _BMI5_std = standardize(_BMI5)


################High blood pressure - categorical feature (1 = no, 2 = yes, 9 = missing)
_RFHYPE5 = extract_feature('_RFHYPE5')
_RFHYPE5[_RFHYPE5 == 9] = np.nan
_RFHYPE5[_RFHYPE5 == 1] = 0
_RFHYPE5[_RFHYPE5 == 2] = 1
_RFHYPE5 = replace_mode(_RFHYPE5)

################High cholesterol - categorical feature (1 = no, 2 = yes, 9 = missing)
_RFCHOL = extract_feature('_RFCHOL')
_RFCHOL[_RFCHOL == 9] = np.nan
_RFCHOL[_RFCHOL == 1] = 0
_RFCHOL[_RFCHOL == 2] = 1
_RFCHOL = replace_mode(_RFCHOL)

################Smoking status - categorical feature (1 = every day, 2 = some days, 3 = formerly, 4 = never, 9 = missing)
_SMOKER3 = extract_feature('_SMOKER3')
_SMOKER3[_SMOKER3 == 9] = np.nan
_SMOKER3 = replace_mode(_SMOKER3)
_SMOKER3, _SMOKER3_mean, _SMOKER3_std = standardize(_SMOKER3)

################Has ever had a stroke  - categorical feature (1 = yes, 2 = no, 7 = don't know, 9 = missing)
CVDSTRK3 = extract_feature('CVDSTRK3')
CVDSTRK3[CVDSTRK3 == 9] = np.nan
CVDSTRK3[CVDSTRK3 == 7] = np.nan
CVDSTRK3[CVDSTRK3 == 2] = 0
CVDSTRK3 = replace_mode(CVDSTRK3)

################Cholesterol checked  - categorical feature (1 = within the last 5 years, 2 = more than 5 years ago, 3 = never, 9 = missing)
_CHOLCHK = extract_feature('_CHOLCHK')
_CHOLCHK[_CHOLCHK == 9] = np.nan
_CHOLCHK = replace_mode(_CHOLCHK)
_CHOLCHK,_ , _ = standardize(_CHOLCHK)

################Has ever had diabetes  - categorical feature (1 = yes, 2 = yes*, 3 = no, 4 = no - pre-diabetes, 7 = don't know, 9 = missing)
DIABETE3 = extract_feature('DIABETE3')
DIABETE3[DIABETE3 == 9] = np.nan
DIABETE3[DIABETE3 == 7] = np.nan
DIABETE3[DIABETE3 == 3] = 0
DIABETE3[DIABETE3 == 4] = 0
DIABETE3[DIABETE3 == 2] = 1
DIABETE3 = replace_mode(DIABETE3)

################Physical activity index  - categorical feature (1 = highly active, 2 = active, 3 = insufficiently active, 4 = inactive, 9 = missing)
_PACAT1 = extract_feature('_PACAT1')
_PACAT1[_PACAT1 == 9] = np.nan
_PACAT1 = replace_mode(_PACAT1)
_PACAT1, _PACAT1_mean, _PACAT1_std = standardize(_PACAT1)

################Total fruits consumed per day  - continuous feature (implied 2 dp)
#_FRUTSUM = extract_feature('_FRUTSUM')

################Total vegetables consumed per day  - continuous feature (implied 2 dp)
#_VEGESUM = extract_feature('_VEGESUM')

################Computed number of drinks of alcohol beverages per week  - continuous feature (99900 = missing)
_DRNKWEK = extract_feature('_DRNKWEK')
_DRNKWEK[_DRNKWEK == 99900] = np.nan
_DRNKWEK = replace_mean(_DRNKWEK)
_DRNKWEK, _DRNKWEK_mean, _DRNKWEK_std = standardize(_DRNKWEK)

################Have any healthcare coverage  - categorical feature (1 = yes, 2 = no, 7 = don't know, 9 = missing)
HLTHPLN1 = extract_feature('HLTHPLN1')
HLTHPLN1[HLTHPLN1 == 9] = np.nan
HLTHPLN1[HLTHPLN1 == 7] = np.nan
HLTHPLN1[HLTHPLN1 == 2] = 0
HLTHPLN1 = replace_mode(HLTHPLN1)

################Could not see doctor because of cost  - categorical feature (1 = yes, 2 = no, 7 = don't know, 9 = missing)
MEDCOST = extract_feature('MEDCOST')
MEDCOST[MEDCOST == 9] = np.nan
MEDCOST[MEDCOST == 7] = np.nan
MEDCOST[MEDCOST == 2] = 0
MEDCOST = replace_mode(MEDCOST)

################General health status  - categorical feature (1 = excellent, 2 = very good, 3 = good, 4 = fair, 5 = poor, 7 = don't know, 9 = missing)
GENHLTH = extract_feature('GENHLTH')
GENHLTH[GENHLTH == 9] = np.nan
GENHLTH[GENHLTH == 7] = np.nan
GENHLTH = replace_mode(GENHLTH)
GENHLTH, GENHLTH_mean, GENHLTH_std = standardize(GENHLTH)

################Number of days mental health not good  - continuous feature (88 = none, 77 = don't know, 99 = refused)
MENTHLTH = extract_feature('MENTHLTH')
MENTHLTH[MENTHLTH == 88] = 0
MENTHLTH[MENTHLTH == 77] = np.nan
MENTHLTH[MENTHLTH == 99] = np.nan
MENTHLTH = replace_mean(MENTHLTH)
MENTHLTH, MENTHLTH_mean, MENTHLTH_std = standardize(MENTHLTH)

################Number of days physical health not good  - continuous feature (88 = none, 77 = don't know, 99 = refused)
PHYSHLTH = extract_feature('PHYSHLTH')
PHYSHLTH[PHYSHLTH == 88] = 0
PHYSHLTH[PHYSHLTH == 77] = np.nan
PHYSHLTH[PHYSHLTH == 99] = np.nan
PHYSHLTH = replace_mean(PHYSHLTH)
PHYSHLTH, PHYSHLTH_mean, PHYSHLTH_std = standardize(PHYSHLTH)

################Difficulty walking or climbing stairs - categorical feature (1 = yes, 2 = no, 7 = don't know, 9 = missing)
DIFFWALK = extract_feature('DIFFWALK')
DIFFWALK[DIFFWALK == 9] = np.nan
DIFFWALK[DIFFWALK == 7] = np.nan
DIFFWALK[DIFFWALK == 2] = 0
DIFFWALK = replace_mode(DIFFWALK)

################Sex - categorical feature (1 = male, 2 = female)
SEX = extract_feature('SEX')
SEX[SEX == 2] = 0
SEX = replace_mode(SEX)

################Age  - categorical feature (1 = 18-24, ... 13 = 80+, 14 = missing)
_AGEG5YR = extract_feature('_AGEG5YR')
_AGEG5YR[_AGEG5YR == 14] = np.nan
_AGEG5YR = replace_mode(_AGEG5YR)
_AGEG5YR, _AGEG5YR_mean, _AGEG5YR_std = standardize(_AGEG5YR)

################Education  - categorical feature (1 = none, ... 6 = college grad, 9 = missing)
EDUCA = extract_feature('EDUCA')
EDUCA[EDUCA == 9] = np.nan
EDUCA = replace_mode(EDUCA)
EDUCA, EDUCA_mean, EDUCA_std = standardize(EDUCA)

################Income level  - categorical feature (1 = low, ... 5 = high, 9 = missing)
_INCOMG = extract_feature('_INCOMG')
_INCOMG[_INCOMG == 9] = np.nan
_INCOMG = replace_mode(_INCOMG)
_INCOMG, _INCOMG_mean, _INCOMG_std = standardize(_INCOMG)

#Here we stack the features together to have the our new X
X = np.hstack((_BMI5.reshape(-1, 1), _RFHYPE5.reshape(-1, 1), _RFCHOL.reshape(-1, 1), _SMOKER3.reshape(-1, 1), CVDSTRK3.reshape(-1, 1), 
               _CHOLCHK.reshape(-1, 1), DIABETE3.reshape(-1, 1), _PACAT1.reshape(-1, 1), # _FRUTSUM.reshape(-1, 1), _VEGESUM.reshape(-1, 1), 
               _DRNKWEK.reshape(-1, 1), HLTHPLN1.reshape(-1, 1), MEDCOST.reshape(-1, 1), GENHLTH.reshape(-1, 1), MENTHLTH.reshape(-1, 1), PHYSHLTH.reshape(-1, 1), 
               DIFFWALK.reshape(-1, 1), SEX.reshape(-1, 1), _AGEG5YR.reshape(-1, 1), EDUCA.reshape(-1, 1), _INCOMG.reshape(-1, 1)))


In [8]:
caracteristics(X, y_train)

Number of x features beeing not nan: 6234565
Number of x features beeing nan: 0
Number of y -1s: 299160
Number of y 1s: 28975
Number of y 0s: 0
Number of y nan: 0


In [9]:
nan_count_per_column = np.sum(np.isnan(X), axis=0)

In [10]:
nan_count_per_column

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
print(X.shape)
print(y_train.shape)

(328135, 19)
(328135,)


We now balance the data

In [13]:
def balance_data(x, y):
    #Balance the data by taking only a certain number of values in y=-1 s.t. the number of y=1 equals the number of y=-1
    
    indices_y_equals_1 = np.where(y == 1)[0]
    indices_y_equals_minus_1 = np.where(y == -1)[0]
    num_positives = np.sum(y == 1)
    num_negatives = np.sum(y == -1)
    selected_indices_neg = np.random.choice(indices_y_equals_minus_1, size=num_positives, replace=False)
    
    selected_indices = np.concatenate((selected_indices_neg, indices_y_equals_1))
    selected_X = x[selected_indices]
    selected_y = y[selected_indices]
    
    return selected_X, selected_y

def balance_data_multiplier(X, y): 
    class_1 = X[y == 1]
    class_minus_1 = X[y == -1]
    count_class_1 = class_1.shape[0]
    count_class_minus_1 = class_minus_1.shape[0]
    num_to_duplicate = count_class_minus_1 - count_class_1
    duplicated_samples = np.tile(class_1, (num_to_duplicate // count_class_1 + 1, 1))[:num_to_duplicate]
    X_balanced = np.vstack((X, duplicated_samples))
    y_balanced = np.hstack((y, np.ones(num_to_duplicate)))
    return X_balanced, y_balanced

#print(y_train.shape)
X_balanced, y_balanced = balance_data_multiplier(X, y_train)
caracteristics(X_balanced, y_balanced)
#X_balanced, y_balanced = balance_data(X, y_train)
#caracteristics(X_balanced, y_balanced)

Number of x features beeing not nan: 11368080
Number of x features beeing nan: 0
Number of y -1s: 299160
Number of y 1s: 299160
Number of y 0s: 0
Number of y nan: 0


Here I just wanted to use the functions used in the exercise sessions. So I change from -1 to 0 for y negative so that I can use them

In [14]:
#Change all the elements with -1 by 0
y_train_working = y_balanced.copy()
y_train_working[y_train_working == -1] = 0
#Make y have the correct shape
y_train_working = y_train_working.reshape(-1, 1)

We split the data into train and val sets

In [15]:
def split_train_val(x, y, k_fold, k):
    k_indices = build_k_indices(y, k_fold) 
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_te = y[te_indice]
    y_tr = y[tr_indice]
    x_te = x[te_indice]
    x_tr = x[tr_indice]
    return x_tr, x_te, y_tr, y_te

X_train, X_val, Y_train, Y_val = split_train_val(X_balanced, y_train_working, 10, 9)
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

(538488, 19)
(59832, 19)
(538488, 1)
(59832, 1)


In [16]:
def drop_nan(X_train, Y_train, X_val, Y_val):
    mask_train = ~np.isnan(X_train).any(axis=1)
    X_train = X_train[mask_train]
    Y_train = Y_train[mask_train]
    
    mask_val = ~np.isnan(X_val).any(axis=1)
    X_val = X_val[mask_val]
    Y_val = Y_val[mask_val]
    return X_train, Y_train, X_val, Y_val

#X_train, Y_train, X_val, Y_val = drop_nan(X_train, Y_train, X_val, Y_val)
caracteristics(X_train, Y_train)
print(X_train.shape)
print(X_val.shape)
print(Y_train.shape)
print(Y_val.shape)

Number of x features beeing not nan: 10231272
Number of x features beeing nan: 0
Number of y -1s: 0
Number of y 1s: 269180
Number of y 0s: 269308
Number of y nan: 0
(538488, 19)
(59832, 19)
(538488, 1)
(59832, 1)


In [17]:
def logistic_regression_gradient_descent(y, x, lambda_, gamma):
    # init parameters
    max_iter = 10000
    threshold = 1e-7
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))
    #print(tx)
    #print(y)

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent_ridge(y, tx, w, gamma, lambda_)
        # log info
        if iter % 100 == 0:
            print("Current iteration={i}, loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return w, loss

In [18]:
gamma = 0.1
lambda_ = 0.0001

w, loss = logistic_regression_gradient_descent(Y_train, X_train, lambda_, gamma)
print(w)

Current iteration=0, loss=0.6931471805599391
Current iteration=100, loss=0.5118641615259965
Current iteration=200, loss=0.49617194660608466
Current iteration=300, loss=0.48977697328248704
Current iteration=400, loss=0.48633679131459223
Current iteration=500, loss=0.48425774041839453
Current iteration=600, loss=0.48290684581378096
Current iteration=700, loss=0.4819771136644689
Current iteration=800, loss=0.4813046046449744
Current iteration=900, loss=0.4807966464513939
Current iteration=1000, loss=0.4803985753059114
Current iteration=1100, loss=0.4800769179488843
Current iteration=1200, loss=0.4798104361117697
Current iteration=1300, loss=0.4795851807451848
Current iteration=1400, loss=0.47939167761696955
Current iteration=1500, loss=0.47922328170250456
Current iteration=1600, loss=0.47907518966749096
Current iteration=1700, loss=0.47894383193053486
Current iteration=1800, loss=0.4788264887544607
Current iteration=1900, loss=0.47872104157340994
Current iteration=2000, loss=0.47862580780

In [28]:
def prediction(tx_test, w):
    compute = sigmoid(np.dot(tx_test, w))
    y_test = (compute >= 0.5).astype(int)
    return y_test
#Now we test the result: % of well classified data
def percentage_well_predicted(true_labels, predicted_labels):
    # Check if both vectors have the same length
    if len(true_labels) != len(predicted_labels):
        raise ValueError("The two vectors must have the same length.")
    # Calculate the number of wrongly predicted points
    num_right = np.sum(true_labels == predicted_labels)
    # Calculate the percentage of wrongly predicted points
    percentage_right = (num_right / len(true_labels)) * 100
    return percentage_right
tx = np.c_[np.ones((X_val.shape[0], 1)), X_val]
print(tx.shape)
print(w.shape)
y_pred_test = prediction(tx, w)
zero_count = np.sum(y_pred_test == 0)
nonzero_count = np.sum(y_pred_test != 0)
print(zero_count)
print(nonzero_count)
print(percentage_well_predicted(Y_val, y_pred_test))

(59832, 20)
(20, 1)
28250
31582
77.63738467709587


Now we want to use our w to predict the results: we first format our x_test values

In [21]:
def extract_feature_test(name):
    filename = 'x_test.csv'
    first_line = np.array(read_first_line(filename))
    index = np.where(first_line == name)
    ind = index[0].item()
    return x_test[:, ind-1]

################Body mass idex - continuous feature
_BMI5 = extract_feature_test('_BMI5')
_BMI5 = replace_mean(_BMI5)
_BMI5, _BMI5_mean, _BMI5_std = standardize(_BMI5)


################High blood pressure - categorical feature (1 = no, 2 = yes, 9 = missing)
_RFHYPE5 = extract_feature_test('_RFHYPE5')
_RFHYPE5[_RFHYPE5 == 9] = np.nan
_RFHYPE5[_RFHYPE5 == 1] = 0
_RFHYPE5[_RFHYPE5 == 2] = 1
_RFHYPE5 = replace_mode(_RFHYPE5)

################High cholesterol - categorical feature (1 = no, 2 = yes, 9 = missing)
_RFCHOL = extract_feature_test('_RFCHOL')
_RFCHOL[_RFCHOL == 9] = np.nan
_RFCHOL[_RFCHOL == 1] = 0
_RFCHOL[_RFCHOL == 2] = 1
_RFCHOL = replace_mode(_RFCHOL)

################Smoking status - categorical feature (1 = every day, 2 = some days, 3 = formerly, 4 = never, 9 = missing)
_SMOKER3 = extract_feature_test('_SMOKER3')
_SMOKER3[_SMOKER3 == 9] = np.nan
_SMOKER3 = replace_mode(_SMOKER3)
_SMOKER3, _SMOKER3_mean, _SMOKER3_std = standardize(_SMOKER3)

################Has ever had a stroke  - categorical feature (1 = yes, 2 = no, 7 = don't know, 9 = missing)
CVDSTRK3 = extract_feature_test('CVDSTRK3')
CVDSTRK3[CVDSTRK3 == 9] = np.nan
CVDSTRK3[CVDSTRK3 == 7] = np.nan
CVDSTRK3[CVDSTRK3 == 2] = 0
CVDSTRK3 = replace_mode(CVDSTRK3)

################Cholesterol checked  - categorical feature (1 = within the last 5 years, 2 = more than 5 years ago, 3 = never, 9 = missing)
_CHOLCHK = extract_feature_test('_CHOLCHK')
_CHOLCHK[_CHOLCHK == 9] = np.nan
_CHOLCHK = replace_mode(_CHOLCHK)
_CHOLCHK,_ , _ = standardize(_CHOLCHK)

################Has ever had diabetes  - categorical feature (1 = yes, 2 = yes*, 3 = no, 4 = no - pre-diabetes, 7 = don't know, 9 = missing)
DIABETE3 = extract_feature_test('DIABETE3')
DIABETE3[DIABETE3 == 9] = np.nan
DIABETE3[DIABETE3 == 7] = np.nan
DIABETE3[DIABETE3 == 3] = 0
DIABETE3[DIABETE3 == 4] = 0
DIABETE3[DIABETE3 == 2] = 1
DIABETE3 = replace_mode(DIABETE3)

################Physical activity index  - categorical feature (1 = highly active, 2 = active, 3 = insufficiently active, 4 = inactive, 9 = missing)
_PACAT1 = extract_feature_test('_PACAT1')
_PACAT1[_PACAT1 == 9] = np.nan
_PACAT1 = replace_mode(_PACAT1)
_PACAT1, _PACAT1_mean, _PACAT1_std = standardize(_PACAT1)

################Total fruits consumed per day  - continuous feature (implied 2 dp)
#_FRUTSUM = extract_feature('_FRUTSUM')

################Total vegetables consumed per day  - continuous feature (implied 2 dp)
#_VEGESUM = extract_feature('_VEGESUM')

################Computed number of drinks of alcohol beverages per week  - continuous feature (99900 = missing)
_DRNKWEK = extract_feature_test('_DRNKWEK')
_DRNKWEK[_DRNKWEK == 99900] = np.nan
_DRNKWEK = replace_mean(_DRNKWEK)
_DRNKWEK, _DRNKWEK_mean, _DRNKWEK_std = standardize(_DRNKWEK)

################Have any healthcare coverage  - categorical feature (1 = yes, 2 = no, 7 = don't know, 9 = missing)
HLTHPLN1 = extract_feature_test('HLTHPLN1')
HLTHPLN1[HLTHPLN1 == 9] = np.nan
HLTHPLN1[HLTHPLN1 == 7] = np.nan
HLTHPLN1[HLTHPLN1 == 2] = 0
HLTHPLN1 = replace_mode(HLTHPLN1)

################Could not see doctor because of cost  - categorical feature (1 = yes, 2 = no, 7 = don't know, 9 = missing)
MEDCOST = extract_feature_test('MEDCOST')
MEDCOST[MEDCOST == 9] = np.nan
MEDCOST[MEDCOST == 7] = np.nan
MEDCOST[MEDCOST == 2] = 0
MEDCOST = replace_mode(MEDCOST)

################General health status  - categorical feature (1 = excellent, 2 = very good, 3 = good, 4 = fair, 5 = poor, 7 = don't know, 9 = missing)
GENHLTH = extract_feature_test('GENHLTH')
GENHLTH[GENHLTH == 9] = np.nan
GENHLTH[GENHLTH == 7] = np.nan
GENHLTH = replace_mode(GENHLTH)
GENHLTH, GENHLTH_mean, GENHLTH_std = standardize(GENHLTH)

################Number of days mental health not good  - continuous feature (88 = none, 77 = don't know, 99 = refused)
MENTHLTH = extract_feature_test('MENTHLTH')
MENTHLTH[MENTHLTH == 88] = 0
MENTHLTH[MENTHLTH == 77] = np.nan
MENTHLTH[MENTHLTH == 99] = np.nan
MENTHLTH = replace_mean(MENTHLTH)
MENTHLTH, MENTHLTH_mean, MENTHLTH_std = standardize(MENTHLTH)

################Number of days physical health not good  - continuous feature (88 = none, 77 = don't know, 99 = refused)
PHYSHLTH = extract_feature_test('PHYSHLTH')
PHYSHLTH[PHYSHLTH == 88] = 0
PHYSHLTH[PHYSHLTH == 77] = np.nan
PHYSHLTH[PHYSHLTH == 99] = np.nan
PHYSHLTH = replace_mean(PHYSHLTH)
PHYSHLTH, PHYSHLTH_mean, PHYSHLTH_std = standardize(PHYSHLTH)

################Difficulty walking or climbing stairs - categorical feature (1 = yes, 2 = no, 7 = don't know, 9 = missing)
DIFFWALK = extract_feature_test('DIFFWALK')
DIFFWALK[DIFFWALK == 9] = np.nan
DIFFWALK[DIFFWALK == 7] = np.nan
DIFFWALK[DIFFWALK == 2] = 0
DIFFWALK = replace_mode(DIFFWALK)

################Sex - categorical feature (1 = male, 2 = female)
SEX = extract_feature_test('SEX')
SEX[SEX == 2] = 0
SEX = replace_mode(SEX)

################Age  - categorical feature (1 = 18-24, ... 13 = 80+, 14 = missing)
_AGEG5YR = extract_feature_test('_AGEG5YR')
_AGEG5YR[_AGEG5YR == 14] = np.nan
_AGEG5YR = replace_mode(_AGEG5YR)
_AGEG5YR, _AGEG5YR_mean, _AGEG5YR_std = standardize(_AGEG5YR)

################Education  - categorical feature (1 = none, ... 6 = college grad, 9 = missing)
EDUCA = extract_feature_test('EDUCA')
EDUCA[EDUCA == 9] = np.nan
EDUCA = replace_mode(EDUCA)
EDUCA, EDUCA_mean, EDUCA_std = standardize(EDUCA)

################Income level  - categorical feature (1 = low, ... 5 = high, 9 = missing)
_INCOMG = extract_feature_test('_INCOMG')
_INCOMG[_INCOMG == 9] = np.nan
_INCOMG = replace_mode(_INCOMG)
_INCOMG, _INCOMG_mean, _INCOMG_std = standardize(_INCOMG)

#Here we stack the features together to have the our new X
X_test = np.hstack((_BMI5.reshape(-1, 1), _RFHYPE5.reshape(-1, 1), _RFCHOL.reshape(-1, 1), _SMOKER3.reshape(-1, 1), CVDSTRK3.reshape(-1, 1), 
               _CHOLCHK.reshape(-1, 1), DIABETE3.reshape(-1, 1), _PACAT1.reshape(-1, 1), # _FRUTSUM.reshape(-1, 1), _VEGESUM.reshape(-1, 1), 
               _DRNKWEK.reshape(-1, 1), HLTHPLN1.reshape(-1, 1), MEDCOST.reshape(-1, 1), GENHLTH.reshape(-1, 1), MENTHLTH.reshape(-1, 1), PHYSHLTH.reshape(-1, 1), 
               DIFFWALK.reshape(-1, 1), SEX.reshape(-1, 1), _AGEG5YR.reshape(-1, 1), EDUCA.reshape(-1, 1), _INCOMG.reshape(-1, 1)))


Now we actually want to predict

In [22]:
def prediction(tx_test, w):
    compute = sigmoid(np.dot(tx_test, w))
    y_test = (compute >= 0.5).astype(int)
    return y_test

In [23]:
tx_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]
print(tx_test)
print(w)
y_pred = prediction(tx_test, w)
print(y_pred)
nonzero_count = np.sum(y_pred != 0)
zero_count = np.sum(y_pred == 0)
print(nonzero_count)
print(zero_count)
y_pred[y_pred == 0] = -1

[[ 1.00000000e+00 -9.09070615e-01  1.00000000e+00 ...  1.54082763e+00
   1.04300780e+00  7.41110652e-01]
 [ 1.00000000e+00 -9.67684643e-01  0.00000000e+00 ... -1.96329906e+00
   1.04300780e+00  7.41110652e-01]
 [ 1.00000000e+00  1.68842320e-15  1.00000000e+00 ...  6.64795955e-01
   1.04300780e+00  7.41110652e-01]
 ...
 [ 1.00000000e+00 -5.13029881e-01  0.00000000e+00 ...  1.54082763e+00
   1.04300780e+00 -6.82577255e-01]
 [ 1.00000000e+00 -6.41347079e-01  0.00000000e+00 ...  1.54082763e+00
   8.50664592e-02  7.41110652e-01]
 [ 1.00000000e+00  2.24024530e+00  1.00000000e+00 ... -5.03246272e-01
   8.50664592e-02  7.41110652e-01]]
[[-1.76984469]
 [ 0.0143856 ]
 [ 0.58026887]
 [ 0.60020169]
 [-0.16170341]
 [ 1.11455573]
 [-0.15362721]
 [ 0.33952425]
 [-0.04014361]
 [-0.03132185]
 [-0.13539887]
 [ 0.23604837]
 [ 0.50991184]
 [ 0.0477477 ]
 [ 0.04622207]
 [ 0.32057781]
 [ 0.76036395]
 [ 0.9025384 ]
 [-0.02587113]
 [-0.07437044]]
[[1]
 [0]
 [0]
 ...
 [0]
 [1]
 [0]]
33043
76336


In [24]:
from helpers import create_csv_submission
create_csv_submission(test_ids, y_pred, "Submission_test_github")